In [294]:
# Open files Neighborhood results, put together, format as nn --> merge with DBSCAN cluster
file = []
folder_location="C:/Data/NNs_"
dir=Path(folder_location)
os.chdir(folder_location)
for f in os.listdir(dir):
    if f.endswith(".csv"):
        df = pd.read_csv(f, sep ="\t")
        df["Pt"]=pd.Series(f).astype("string")[0].split(".")[-2]
        df_dcs = df[~df.Init_Phenotype.isin(["Others", "CD8+"])]
        file.append(df_dcs)
        df_nns = pd.concat(file)
df_nns.to_csv("C:/Data/DCs_results/DCs_nn.txt")
df_nns_re = df_nns.drop(columns=["XY", "Unnamed: 0","Unnamed: 0.1" ])
df_nns_use = df_nns_re.rename(columns={"InitXY":"XY", "Init_Phenotype":"Phenotype"})
nns_counts = df_nns_use.pivot_table(index=["Pt", "XY", "Phenotype"], columns=["Neighbor_Phenotype"],
values=["Neighbor_Phenotype"], aggfunc=lambda x:len(x))
nns_counts.reset_index(inplace=True)
nns_counts["Pt2"]=nns_counts["Pt"].apply(lambda x:x.split("_")[-1])
nns = nns_counts.drop(columns="Pt")
nns_ = nns.rename(columns={"Pt2":"Pt"})
nns_["X"]=nns_["XY"].apply(lambda x: x.replace("(","").replace(")","").split(",")[0])
nns_["Y"]=nns_["XY"].apply(lambda x: x.replace("(","").replace(")","").split(",")[1])
nns_2 = nns_.drop(columns="XY")
nns_2.columns =['Phenotype', 'CD141+PD-L1+', 'CD141+PD-L1-', 'CD1a+', 'CD1a+PD-L1+','CD1a+PD-L1-', 'CD8+', 'Others', 'Pt', 'X', 'Y']
nn3=nns_2[["Pt",'Phenotype', 'CD141+PD-L1+', 'CD141+PD-L1-', 'CD1a+', 'CD1a+PD-L1+','CD1a+PD-L1-', 'CD8+', 'Others']]
nn3["X"]=nns_2["X"].astype("float64")
nn3["Y"]=nns_2["Y"].astype("float64")

# Merge nn with cluster:
df_DCs = pd.read_csv("C:/Data/DCs1_res/Res_all.txt")
df_DCs_nn = df_DCs[["Pt", "Phenotype", "Cluster", "X", "Y"]].dropna(subset=["Cluster"])
df_DCs_nn_use_ = df_DCs_nn[~df_DCs_nn["Phenotype"].isin(["CD8+","Others"])]
df_DCs_nn_use_2=df_DCs_nn_use_[['Pt', 'Phenotype', 'Cluster']]
df_DCs_nn_use_2["X"]=df_DCs_nn_use_["X"].astype("float64")
df_DCs_nn_use_2["Y"]=df_DCs_nn_use_["Y"].astype("float64")

# Fix inconsistent sample name, --> merge
Pt1=nn3["Pt"].unique()
df_pt1=pd.DataFrame(Pt1)
Pt2=df_DCs_nn_use_2["Pt"].unique()
df_pt2=pd.DataFrame(Pt2)
names = pd.concat([df_pt1,df_pt2],axis=1) 

for p, v in zip(names.iloc[:,0], names.iloc[:,1]): 
    df_DCs_nn_use_2.loc[(df_DCs_nn_use_2["Pt"]==v),"Pt"]=p

# merge
i_files=[]
for pts in nn3.Pt.unique():
    i_files.append(pd.merge(nn3[(nn3.Pt==pts)],df_DCs_nn_use_2[(df_DCs_nn_use_2.Pt==pts)]))
all_files=pd.concat(i_files)
all_files.to_csv("C:/Data/DCs_results/nn_DCs_cluster.txt")

# Add the type of clusters to the merge dataset
# append cluster type
all_files["Pt"]=all_files["Pt"].astype("string")
all_files["Cluster"]=all_files["Cluster"].astype("int64")
df_Cluster_pv2 = pd.read_csv("C:/Data/DCs_results/c_Type.txt")
cluster_type = df_Cluster_pv2[["Pt", "Cluster", "c_Type"]]
cluster_type["Pt"]=cluster_type["Pt"].astype("string")
cluster_type["Cluster"]=cluster_type["Cluster"].astype("int64")

# fix the names
PtA=all_files["Pt"].unique()
df_PtA=pd.DataFrame(PtA)
PtB=cluster_type["Pt"].unique()
df_PtB=pd.DataFrame(PtB)
names_ = pd.concat([df_PtA,df_PtB],axis=1) 
for p, v in zip(names_.iloc[:,0], names_.iloc[:,1]): 
    cluster_type.loc[(cluster_type["Pt"]==v),"Pt"]=p
   
all_files_cType = pd.merge(all_files, cluster_type, how="outer")
all_files_cType.to_csv("C:/Data/DCs_results/nn_DCs_clustera_cType.txt")


# Plot neighborhoods #, comparing dendritic cells iso, homocluster, heterocluster
cDC1_all_files_cType_spec=all_files_cType[["Pt","Phenotype","c_Type","CD141+PD-L1+","CD141+PD-L1-","CD1a+","CD1a+PD-L1+","CD1a+PD-L1-","CD8+","Others"]].fillna(0)
cDC1_all_files_cType_spec["cDC1"] = cDC1_all_files_cType_spec["CD141+PD-L1+"] + cDC1_all_files_cType_spec["CD141+PD-L1-"]
cDC1_all_files_cType_spec["CD1A"] = cDC1_all_files_cType_spec["CD1a+"] + cDC1_all_files_cType_spec["CD1a+PD-L1+"] + cDC1_all_files_cType_spec["CD1a+PD-L1-"] 
cDC1_all_files_cType_spec["Other"] = cDC1_all_files_cType_spec["CD8+"] + cDC1_all_files_cType_spec["Others"]
cDC1_all_files_cType_sim = cDC1_all_files_cType_spec[["Pt","Phenotype", "c_Type", "cDC1", "CD1A", "Other"]]
cDC1_all_files_cType_bc = cDC1_all_files_cType_sim[~cDC1_all_files_cType_sim.Pt.isin(["Norm LN1x4","Norm LN2x4","Norm LN3x4","Norm LN4x4"])]
cDC1_all_files_cType_perbc = cDC1_all_files_cType_bc.pivot_table(index=["Pt","Phenotype","c_Type"], aggfunc="mean")
cDC1_all_files_cType_perbc.reset_index(inplace = True)
cDC1_all_files_cType_perbc_simpleDC = cDC1_all_files_cType_perbc.replace({"Phenotype":{"CD141+PD-L1+":"cDC1", "CD141+PD-L1-":"cDC1",
                                                "CD1a+PD-L1+":"CD1A", "CD1a+PD-L1-":"CD1A",
                                                "CD1a+":"CD1A"}})
data_plot = pd.melt(cDC1_all_files_cType_perbc_simpleDC, id_vars=['Pt',"Phenotype",'c_Type'], value_vars = ['CD1A', 'cDC1','Other'])
data_plot2 = data_plot.replace({"c_Type":{"iso":"Iso"}})


# plot for cDC1 (iso, heterocluster, homocluster) and how many neighbors they have
sns.set(style = "ticks", font_scale = 1.5, color_codes=True)
sns.set_context("talk")
neighbor_for_cDC1 = data_plot2[(data_plot2.Phenotype=="cDC1")]
g= sns.catplot(data= neighbor_for_cDC1, kind="bar", height = 6, aspect = .75, 
               x="c_Type", y= "value", hue="variable", palette=("crimson","deepskyblue","gainsboro"), edgecolor="0.1")
plt.xlabel('Type of DC clusters', fontsize=17)
plt.ylabel('Neighbor Cell Count',fontsize=17)
g._legend.remove()
g.savefig('C:/Data/DCs_neighbors/Neighbor_for_cDC1.tiff', format='tiff', dpi=600)
plt.show()

# plot for CD1A  (iso, heterocluster, homocluster) and how many neighbors they have
sns.set(style = "ticks", font_scale = 1.5, color_codes=True)
sns.set_context("talk")
neighbor_for_CD1A = data_plot2[(data_plot2.Phenotype=="CD1A")].replace({"variable":{"CD1A":"CD1a+", "Other":"other types"}})
g= sns.catplot(data= neighbor_for_CD1A, kind="bar", height = 6, aspect = .75, 
               x="c_Type", y= "value", order=["Het", "CD1a-h", "Iso"], hue="variable", palette=("crimson","deepskyblue","gainsboro"), edgecolor="0.1")
plt.xlabel('Type of DC clusters', fontsize=17)
plt.ylabel('Neighbor Cell Count',fontsize=17)
plt.legend(title="Neighbor Cell", bbox_to_anchor=(1.05, 1), borderaxespad=0.)
g._legend.remove()
g.savefig('C:/Data/DCs_neighbors/Neighbor_for_CD1A.tiff', format='tiff', dpi=600)
plt.show()

# Statistical analysis
# p value having CD1A neighbors for cDC1 (Het vs. Iso) 
cDC1_neighborCD1A = neighbor_for_cDC1[(neighbor_for_cDC1.variable=="CD1A")][["Pt","c_Type","value"]].pivot_table(index=["Pt","c_Type"], aggfunc="mean")
cDC1_neighborCD1A.reset_index(inplace = True)
Het = cDC1_neighborCD1A[(cDC1_neighborCD1A.c_Type=="Het")]
Iso = cDC1_neighborCD1A[(cDC1_neighborCD1A.c_Type=="Iso")]
dHetIso=pd.merge(Het, Iso, on="Pt")
p_cDC1_neibrCD1A_HetvsIso = wilcoxon(dHetIso.value_x,dHetIso.value_y)[1]

# p value having cDC1 neighbors for CD1A (Het vs. Iso) 
CD1A_Het_neighborcDC1 = neighbor_for_CD1A[(neighbor_for_CD1A.c_Type=="Het") & (neighbor_for_CD1A.variable=="cDC1")][["Pt","value"]].pivot_table(index=["Pt"], aggfunc="mean").iloc[:,0].to_list()
CD1A_Iso_neighborcDC1 = neighbor_for_CD1A[(neighbor_for_CD1A.c_Type=="Iso") & (neighbor_for_CD1A.variable=="cDC1")][["Pt","value"]].pivot_table(index=["Pt"], aggfunc="mean").iloc[:,0].to_list()
p_CD1A_neibrcDC1_HetvsIso = wilcoxon(CD1A_Het_neighborcDC1,CD1A_Iso_neighborcDC1)[1]
CD1A_neighborcDC1 = neighbor_for_CD1A[(neighbor_for_CD1A.variable=="cDC1")][["Pt","c_Type","value"]].pivot_table(index=["Pt","c_Type"], aggfunc="mean")
CD1A_neighborcDC1.reset_index(inplace = True)
Het1a = CD1A_neighborcDC1[(CD1A_neighborcDC1.c_Type=="Het")]
Iso1a = CD1A_neighborcDC1[(CD1A_neighborcDC1.c_Type=="Iso")]
dHetIso1a=pd.merge(Het1a, Iso1a, on="Pt")
p_CD1A_neibrcDC1_HetvsIso = wilcoxon(dHetIso1a.value_x,dHetIso.value_y)[1]

wilcoxon = pd.DataFrame({"cDC1 have CD1A neibor, iso vs Het":p_cDC1_neibrCD1A_HetvsIso, "CD1A have cDC1 neibor, iso vs Het":[p_CD1A_neibrcDC1_HetvsIso]})
wilcoxon.to_csv("C:/Data/DCs_neighbors/Wilcoxon_pvalue.txt")


# Plot in Heterclusters, size of clusster vs. neighbor change (per Pts)
SIZE_cType_Pts= pd.read_csv("C:/Data/DCs_results/SIZE_cType_Pts.txt")
SIZE_Het = SIZE_cType_Pts.rename(columns={"Unnamed: 0":"Pt"})[["Pt","Het_size"]].replace({"Pt":{"20190222_RZ_DC PD-L1 Panel_12195-23 LNNC2x6":"12195-23 LNNC2x6",
                                                                                               "20190222_RZ_DC PD-L1 Panel_12195-09 LNNA1x5":"12195-09 LNNA1x5",
                                                                                               "20190222_RZ_DC PD-L1 Panel_12195-13 LNNA1x5":"12195-13 LNNA1x5",
                                                                                               "20190222_RZ_DC PD-L1 Panel_14346-1-37 C1x3":"14346-1-37 C1x3",
                                                                                               "20190222_RZ_DC PD-L1 Panel_14346-1-23 (Block #A2)x4":"14346-1-23 (Block #A2)x4",
                                                                                               "20190222_RZ_DC PD-L1 Panel_14346-1-28 (Block #B1)x6":"14346-1-28 (Block #B1)x6"}})
Pt_Het_cDC1_neighborCD1A = neighbor_for_cDC1[(neighbor_for_cDC1.c_Type=="Het") & (neighbor_for_cDC1.variable=="CD1A")][["Pt","value"]].pivot_table(index=["Pt"], aggfunc="mean")
Pt_Het_CD1A_neighborcDC1 = neighbor_for_CD1A[(neighbor_for_CD1A.c_Type=="Het") & (neighbor_for_CD1A.variable=="cDC1")][["Pt","value"]].pivot_table(index=["Pt"], aggfunc="mean")
First_merge = pd.merge(SIZE_Het,Pt_Het_cDC1_neighborCD1A,on=["Pt"], how="outer").rename(columns={"value":"cDC1_#nbr_CD1A"})
Second_merge = pd.merge(First_merge, Pt_Het_CD1A_neighborcDC1, on=["Pt"], how="outer").rename(columns={"value":"CD1A_#nbr_cDC1"}).dropna()

# Plot CD1A Het vs. neighbor cDC1
sns.set(style = "ticks", font_scale = 1.5, color_codes=True)
sns.set_context("talk")
g = sns.regplot(data=Second_merge , x="Het_size", y="CD1A_#nbr_cDC1", scatter_kws={"s": 50, "color":"deepskyblue","edgecolor":"black"},
               line_kws={"color":"deepskyblue"})
plt.xlim(-10, 6000)
plt.ylim(-0.1, 1.2)

plt.xlabel('Cluster Size (Total Cells)', fontsize=17)
plt.ylabel('Neighbor Cell Count',fontsize=17)
plt.tight_layout()
g.figure.savefig('C:/Data/DCs_neighbors/HetSize_vs_CD1A_neiborcDC1.tiff', format='tiff', dpi=600)
plt.show()


# Plot cDC1 Het vs. neighbor CD1A
sns.set(style = "ticks", font_scale = 1.5, color_codes=True)
sns.set_context("talk")
g = sns.regplot(data=Second_merge , x="Het_size", y="cDC1_#nbr_CD1A", scatter_kws={"s": 50, "color":"crimson","edgecolor":"black"},
               line_kws={"color":"crimson"})
plt.xlim(-10, 6000)
plt.ylim(-0.1, 1.2)

plt.xlabel('Cluster Size (Total Cells)', fontsize=17)
plt.ylabel('Neighbor Cell Count',fontsize=17)
plt.tight_layout()
g.figure.savefig('C:/Data/DCs_neighbors/HetSize_vs_cDC1_neiborCD1A.tiff', format='tiff', dpi=600)
plt.show()

## Pearson's correlation 
p_CD1A_nbr_cDC1 = pearsonr(x=Second_merge["Het_size"].to_list(), y=Second_merge["CD1A_#nbr_cDC1"].to_list())[1]
r_CD1A_nbr_cDC1 = pearsonr(x=Second_merge["Het_size"].to_list(), y=Second_merge["CD1A_#nbr_cDC1"].to_list())[0]
p_cDC1_nbr_CD1A = pearsonr(x=Second_merge["Het_size"].to_list(), y=Second_merge["cDC1_#nbr_CD1A"].to_list())[1]
r_cDC1_nbr_CD1A = pearsonr(x=Second_merge["Het_size"].to_list(), y=Second_merge["cDC1_#nbr_CD1A"].to_list())[0]
pearson = pd.DataFrame({"Het size vs cDC1 have CD1A neibor":p_cDC1_nbr_CD1A, "Het size vs CD1A have cDC1 neibor":[p_CD1A_nbr_cDC1],
                       "Het size vs cDC1 have CD1A neibor_r":r_cDC1_nbr_CD1A, "Het size vs CD1A have cDC1 neibor_r":[r_CD1A_nbr_cDC1]})
pearson.to_csv("C:/Data/DCs_neighbors/pearson_pvalue.txt")